### Data Generating Process of the sales data

Let $n$ be the number of clients. We define :

- $\Lambda_i \sim \Gamma(a,b) \forall i $
- $nb\_purchases_i | \Lambda_i \sim \mathcal{P}(\lambda_i) \quad \forall i$
- $avg\_purchase \sim \mathcal{LN}(\mu,\sigma) \quad \forall i$

In [87]:
import numpy as np
import pandas as pd
import scipy
from datetime import datetime, timedelta
print("Test : les versions suivantes sont utilisées :",pd.__version__, np.__version__, scipy.__version__)

Test : les versions suivantes sont utilisées : 2.3.3 1.25.2 1.11.4


In [88]:
# =====================
# Parameters
# =====================
n = 15000
T = 26
y_start = 2020
y_end = y_start + T
y_current = pd.Timestamp.now().year

np.random.seed(42)

mu_unit = np.log(50) - 0.32
sigma_unit = 0.8

shape = 2.0
scale = 1.0

sigma_satisfaction = 0.5

# Satisfaction → basket strength
a_value = mu_unit
b_value = 0.3

# =====================
# Client IDs
# =====================
client_ids = np.array([f"C-{i:05d}" for i in range(1, n + 1)])

# =====================
# Latent heterogeneity (stable across years)
# =====================
lambda_clients = np.random.gamma(shape=shape, scale=scale, size=n)

# =====================
# Storage
# =====================
all_purchases = {client_id: {} for client_id in client_ids}
records = []

# =====================
# Simulation
# =====================
for year in range(y_start, y_end):

    # Frequency
    nb_purchases = np.random.poisson(lam=lambda_clients, size=n)

    # Satisfaction
    satisfaction_raw = (
        nb_purchases * np.random.uniform(0, 5, size=n)
    )
    yearly_satisfaction = np.minimum(satisfaction_raw, 5)

    # Basket value depends on satisfaction
    avg_purchase = np.random.lognormal(
        mean=a_value + b_value * yearly_satisfaction,
        sigma=sigma_unit,
        size=n
    )

    # Total value
    tot_purchases = nb_purchases * avg_purchase

    # Purchase dates
    for i in range(n):
        client_id = client_ids[i]

        if nb_purchases[i] > 0:
            all_days = pd.date_range(
                start=f"{year}-01-01",
                end=f"{year}-12-31",
                freq="D"
            )
            dates = np.random.choice(
                all_days,
                size=nb_purchases[i],
                replace=True  # multiple purchases per day allowed
            )
            all_purchases[client_id][year] = dates
        else:
            all_purchases[client_id][year] = np.array([])

    # Churn
    churn = np.random.binomial(
        1,
        1 / (1 + 8 * yearly_satisfaction),
        size=n
    )

    # Panel slice
    year_df = pd.DataFrame({
        "Client_ID": client_ids,
        "Year": year,
        "Nb_Purchases": nb_purchases,
        "Tot_Purchases": tot_purchases,
        "Yearly_Satisfaction": yearly_satisfaction,
        "Churn": churn,
    })

    records.append(year_df)

# =====================
# Build panel
# =====================
panel_data = pd.concat(records, ignore_index=True)

# =====================
# Build long purchase table
# =====================
rows = []

for client_id, years in all_purchases.items():
    for year, dates in years.items():
        for d in dates:
            rows.append([client_id, year, pd.Timestamp(d)])

purchases_df = pd.DataFrame(
    rows,
    columns=["Client_ID", "Year", "Purchase_Date"]
)

# =====================
# Aggregates
# =====================
last = purchases_df.groupby(["Client_ID", "Year"])["Purchase_Date"].max()
first = purchases_df.groupby(["Client_ID", "Year"])["Purchase_Date"].min()
first_alltime = purchases_df.groupby("Client_ID")["Purchase_Date"].min()

# Shift first purchase of each year to previous year
first_next = (
    first
    .reset_index()
    .assign(Year=lambda x: x["Year"] - 1)
    .rename(columns={"Purchase_Date": "First_Purchase_Next"})
)

# =====================
# Merge into panel
# =====================
panel_data = (
    panel_data
    .merge(
        last.reset_index().rename(columns={"Purchase_Date": "Last_Purchase"}),
        on=["Client_ID", "Year"],
        how="left"
    )
    .merge(
        first_next,
        on=["Client_ID", "Year"],
        how="left"
    )
    .merge(
        first_alltime.reset_index().rename(
            columns={"Purchase_Date": "First_Purchase_AllTime"}
        ),
        on="Client_ID",
        how="left"
    )
)

panel_data.loc[panel_data["Churn"] == 1, "First_Purchase_Next"] = pd.NaT

# =====================
# Recency
# =====================
fallback = pd.to_datetime(panel_data["Year"] + 2, format="%Y")

panel_data["Recency"] = (
    panel_data["First_Purchase_Next"].fillna(fallback)
    - panel_data["Last_Purchase"]
).dt.days

# =====================
# Tenure (NEW)
# TENURE(client, year) = 01/01/(year+1) - first_purchase_all_time
# =====================
tenure_ref = pd.to_datetime(panel_data["Year"] + 1, format="%Y")

panel_data["Tenure"] = (
    tenure_ref - panel_data["First_Purchase_AllTime"]
).dt.days

# =====================
# Drop last year (no recency possible)
# =====================
panel_data = panel_data[panel_data["Year"] < y_end - 1]

# =====================
# Remove data after churn
# =====================
panel_data = panel_data.sort_values(["Client_ID", "Year"])

panel_data["Has_Churned"] = (
    panel_data
    .groupby("Client_ID")["Churn"]
    .cumsum()
)

panel_data["Period_since_churn"] = (
    panel_data
    .groupby("Client_ID")["Has_Churned"]
    .cumsum()
)

final_data = (
    panel_data[panel_data["Period_since_churn"] <= 1]
    .drop(columns=["Has_Churned", "Period_since_churn"])
)


KeyboardInterrupt: 

In [ ]:
# Example: show data for some clients

# For 10 clients
pd.set_option('display.max_rows', None)
print(final_data.loc[final_data['Client_ID'].isin([f"C-{i:05d}" for i in range(1, 11)])].sort_values(by=['Client_ID', 'Year']))
pd.reset_option('display.max_rows')
print(final_data.head())
print(final_data.describe())

# For 1 client in particular
print(final_data[final_data['Client_ID']=='C-01050'])

      Client_ID  Year  Nb_Purchases  Tot_Purchases  Yearly_Satisfaction  \
0       C-00001  2020             4     421.877877             5.000000   
15000   C-00001  2021             2     287.682213             5.000000   
30000   C-00001  2022             4    1048.807532             5.000000   
45000   C-00001  2023             2      39.947007             0.433520   
1       C-00002  2020             2     234.262360             4.004101   
15001   C-00002  2021             2      40.542243             4.541208   
30001   C-00002  2022             2     272.401470             5.000000   
45001   C-00002  2023             3     611.639802             5.000000   
60001   C-00002  2024             5     973.587985             5.000000   
75001   C-00002  2025             2     228.998299             5.000000   
90001   C-00002  2026             0       0.000000             0.000000   
2       C-00003  2020             1      46.644043             1.990212   
15002   C-00003  2021    

In [ ]:
# Export if consistent

# Export to csv
final_data.to_csv(
    r"C:\Users\fquet\Documents\TRAVAIL\Travail_personnel\Projet - Vuciliette\src\data\sales.csv",
    index=False
)

In [ ]:
# Dataset de target : CLV per client
CLV_data = final_data.groupby("Client_ID").agg({
    "Tot_Purchases": "sum"}).rename(columns={"Tot_Purchases": "CLV"})
print(CLV_data.head())
CLV_data.to_csv(
    r"C:\Users\fquet\Documents\TRAVAIL\Travail_personnel\Projet - Vuciliette\src\data\CLV_data.csv",
    index=True)
print(final_data.head(10))

# Training data set : final_data for a single year 
sales_2025= final_data[final_data["Year"]==2025]
sales_2025.head(10)
sales_2025.describe()
sales_2025.to_csv(
    r"C:\Users\fquet\Documents\TRAVAIL\Travail_personnel\Projet - Vuciliette\src\data\sales_2025.csv",
    index=False
)   

                   CLV
Client_ID             
C-00001    1798.314629
C-00002    2361.432159
C-00003    2015.981331
C-00004     361.343821
C-00005    3246.027013
      Client_ID  Year  Nb_Purchases  Tot_Purchases  Yearly_Satisfaction  \
0       C-00001  2020             4     421.877877             5.000000   
15000   C-00001  2021             2     287.682213             5.000000   
30000   C-00001  2022             4    1048.807532             5.000000   
45000   C-00001  2023             2      39.947007             0.433520   
1       C-00002  2020             2     234.262360             4.004101   
15001   C-00002  2021             2      40.542243             4.541208   
30001   C-00002  2022             2     272.401470             5.000000   
45001   C-00002  2023             3     611.639802             5.000000   
60001   C-00002  2024             5     973.587985             5.000000   
75001   C-00002  2025             2     228.998299             5.000000   

       Churn 